# Tutorial

If you launched this using the "template" launcher in the Janelia jupyterhub instance, there may be lots of extra whitespace in the cells below.

(That should be fixed with the next JupyterHub update.)

For now, you'll be better off copying the template yourself, and using that copy of the notebook:

In [ ]:
%%bash
cp /misc/sc/jupyterhub/etc/jupyter/templates/ngsidekick-tutorial/tutorial.ipynb ~/ngsidekick-tutorial.ipynb

In [1]:
import os
import copy
import json
import numpy as np
import pandas as pd

import ngsidekick as ngsk

In [2]:
# Some cells below will write their outputs to this directory.
TUTORIAL_EXPORTS = 'ngsidekick-tutorial-exports'
os.makedirs(TUTORIAL_EXPORTS, exist_ok=True)

# Neuroglancer State

Neuroglancer links encode a JSON object that captures nearly all viewer state: layer visibility, camera orientation, etc.

[1]: https://neuroglancer-demo.appspot.com/#!gs://flyem-male-cns/v0.9/male-cns-v0.9.json

In [3]:
template_state = ngsk.parse_nglink('https://neuroglancer-demo.appspot.com/#!gs://flyem-user-links/short/ngsidekick-tutorial-example-scene-1.json')

Most of the top-level keys control the global viewer settings: Coordinate space, camera orientation, zoom level, panel layout, etc.

In [4]:
template_state

{'title': 'Example Scene 1',
 'dimensions': {'x': [8e-09, 'm'], 'y': [8e-09, 'm'], 'z': [8e-09, 'm']},
 'position': [48311.5, 25295.765625, 28511.5],
 'crossSectionScale': 88.13122564876332,
 'projectionOrientation': [0, -0.002181659685447812, 0, 0.999997615814209],
 'projectionScale': 99656.51830908704,
 'layers': [{'type': 'image',
   'source': {'url': 'precomputed://gs://flyem-male-cns/em/em-clahe-jpeg',
    'subsources': {'default': True},
    'enableDefaultSubsources': False},
   'tab': 'rendering',
   'name': 'em-clahe'},
  {'type': 'segmentation',
   'source': {'url': 'precomputed://gs://flyem-male-cns/v0.9/segmentation',
    'subsources': {'default': True, 'properties': True, 'mesh': True},
    'enableDefaultSubsources': False},
   'tab': 'segments',
   'segments': [],
   'name': 'cns-seg'}],
 'showSlices': False,
 'selectedLayer': {'flex': 1.57,
  'size': 415,
  'visible': True,
  'layer': 'cns-seg'},
 'layout': '4panel-alt'}

Neuroglancer stores the layers as a list (to preserve their order), but for programmatic manipulation it's convenient to access them by their `name`.  Here's a convenience function for that:

In [5]:
state = copy.deepcopy(template_state)
seg_layer = ngsk.layer_state(state, 'cns-seg')

In [6]:
# Select two neurons.
seg_layer['segments'] = ["10009", "10157"]

A (non-shortened) neuroglancer link includes the complete JSON state directly in the URL.  (It might be pretty long!)

To turn our state into a URL, just URL-encode the JSON text and give it an appropriate prefix.

In [7]:
import urllib

encoded_state = urllib.parse.quote(json.dumps(state))

print("https://neuroglancer-demo.appspot.com/#!" + encoded_state)

https://neuroglancer-demo.appspot.com/#!%7B%22title%22%3A%20%22Example%20Scene%201%22%2C%20%22dimensions%22%3A%20%7B%22x%22%3A%20%5B8e-09%2C%20%22m%22%5D%2C%20%22y%22%3A%20%5B8e-09%2C%20%22m%22%5D%2C%20%22z%22%3A%20%5B8e-09%2C%20%22m%22%5D%7D%2C%20%22position%22%3A%20%5B48311.5%2C%2025295.765625%2C%2028511.5%5D%2C%20%22crossSectionScale%22%3A%2088.13122564876332%2C%20%22projectionOrientation%22%3A%20%5B0%2C%20-0.002181659685447812%2C%200%2C%200.999997615814209%5D%2C%20%22projectionScale%22%3A%2099656.51830908704%2C%20%22layers%22%3A%20%5B%7B%22type%22%3A%20%22image%22%2C%20%22source%22%3A%20%7B%22url%22%3A%20%22precomputed%3A//gs%3A//flyem-male-cns/em/em-clahe-jpeg%22%2C%20%22subsources%22%3A%20%7B%22default%22%3A%20true%7D%2C%20%22enableDefaultSubsources%22%3A%20false%7D%2C%20%22tab%22%3A%20%22rendering%22%2C%20%22name%22%3A%20%22em-clahe%22%7D%2C%20%7B%22type%22%3A%20%22segmentation%22%2C%20%22source%22%3A%20%7B%22url%22%3A%20%22precomputed%3A//gs%3A//flyem-male-cns/v0.9/segmentation

There's also a convenience function in ngsk for that:

In [8]:
ngsk.encode_ngstate('https://neuroglancer-demo.appspot.com', state)

'https://neuroglancer-demo.appspot.com/#!%7B%22title%22%3A%20%22Example%20Scene%201%22%2C%20%22dimensions%22%3A%20%7B%22x%22%3A%20%5B8e-09%2C%20%22m%22%5D%2C%20%22y%22%3A%20%5B8e-09%2C%20%22m%22%5D%2C%20%22z%22%3A%20%5B8e-09%2C%20%22m%22%5D%7D%2C%20%22position%22%3A%20%5B48311.5%2C%2025295.765625%2C%2028511.5%5D%2C%20%22crossSectionScale%22%3A%2088.13122564876332%2C%20%22projectionOrientation%22%3A%20%5B0%2C%20-0.002181659685447812%2C%200%2C%200.999997615814209%5D%2C%20%22projectionScale%22%3A%2099656.51830908704%2C%20%22layers%22%3A%20%5B%7B%22type%22%3A%20%22image%22%2C%20%22source%22%3A%20%7B%22url%22%3A%20%22precomputed%3A//gs%3A//flyem-male-cns/em/em-clahe-jpeg%22%2C%20%22subsources%22%3A%20%7B%22default%22%3A%20true%7D%2C%20%22enableDefaultSubsources%22%3A%20false%7D%2C%20%22tab%22%3A%20%22rendering%22%2C%20%22name%22%3A%20%22em-clahe%22%7D%2C%20%7B%22type%22%3A%20%22segmentation%22%2C%20%22source%22%3A%20%7B%22url%22%3A%20%22precomputed%3A//gs%3A//flyem-male-cns/v0.9/segmentatio

It can be cumbersome to share long links, and not all standard link shorteners can handle neuroglancer links with complete fidelity. Fortunately, neuroglancer can also unpack the JSON state from a JSON file (as shown above in the link we used to load the initial state).  However, that requires hosting the JSON file somewhere, either over `https` or in a Google Storage bucket (prefixed with `gs://`).

Janelia FlyEM offers a [neuroglancer link shortener][shortng] that stores your link in a GCS bucket (with caveats).  In addition to the Web UI, it has a REST API as shown here.

[shortng]: https://shortng-bmcp5imp6q-uc.a.run.app/shortener.html

(If you prefer to use your own GCS bucket, see the convenience functions in [`ngsidekick.gcs`][gcs].)

[gcs]: https://janelia-flyem.github.io/ngsidekick/docs/gcs.html

In [9]:
import requests

SHORTENER_URL = 'https://shortng-bmcp5imp6q-uc.a.run.app/shortng'

r = requests.post(f'{SHORTENER_URL}', json={
        'state': state,

        'domain': 'https://neuroglancer.janelia.org',

        # Optional: A default name is chosen if you don't provide one.
        'filename': 'my-state.json',

        # Optional: A password to prevent others from writing to the same filename.
        # Note: Does NOT prevent others from using your link!
        'password': 'my-password',
    })

print(r.json())

{'link': 'https://neuroglancer.janelia.org#!gs://flyem-user-links/short/my-state.json'}


# Download Example data

MaleCNS neuron properties and soma locations.

Obtained from the [MaleCNS download page.][1]

[1]: https://male-cns.janelia.org/download

In [10]:
body_annotations_url = 'https://storage.googleapis.com/flyem-male-cns/v0.9/connectome-data/flat-connectome/body-annotations-male-cns-v0.9-minconf-0.5.feather'
df = pd.read_feather(body_annotations_url)

# The "type" column has the cell type, but later on we'll use "type" to mean something else,
# so rename "type" -> "cell_type"
df = df.rename(columns={'type': 'cell_type'})

# Make the index correspond to the body ID
df.index = df['bodyId'].rename('segment')

print(df.columns)
df.head()

Index(['bodyId', 'flywireType', 'group', 'hemibrainType', 'instance',
       'itoleeHl', 'somaSide', 'statusLabel', 'superclass', 'supertype',
       'cell_type', 'class', 'entryNerve', 'fruDsx', 'matchingNotes',
       'rootSide', 'assignedOlHex1', 'assignedOlHex2', 'mancBodyid',
       'mancGroup', 'mancType', 'somaNeuromere', 'subclass', 'trumanHl',
       'dimorphism', 'synonyms', 'birthtime', 'mancSerial', 'mcnsSerial',
       'exitNerve', 'serialMotif', 'receptorType', 'somaLocation',
       'tosomaLocation', 'status'],
      dtype='object')


,bodyId,flywireType,group,hemibrainType,instance,itoleeHl,somaSide,statusLabel,superclass,supertype,...,synonyms,birthtime,mancSerial,mcnsSerial,exitNerve,serialMotif,receptorType,somaLocation,tosomaLocation,status
segment,,,,,,,,,,,,,,,,,,,,,
10001,10001,DNp01,10001.0,Giant Fiber,DNp01(GF)_R,putative_primary,R,Roughly traced,descending_neuron,None,...,Kennedy and Broadie 2018: GF,early,NaN,NaN,None,None,None,"[37124, 22258, 36274]",None,Traced
10002,10002,OCG01d,10002.0,OCG01,OCG01d_L,None,L,Roughly traced,visual_projection,None,...,None,None,NaN,NaN,None,None,None,"[50214, 10279, 27497]",None,Traced
10003,10003,VCH,10003.0,VCH,VCH_R,putative_primary,R,Prelim Roughly traced,visual_centrifugal,None,...,None,early,NaN,NaN,None,None,None,"[46716, 21029, 15714]",None,Traced
10005,10005,AOTU019,10005.0,AOTU019,AOTU019_R,putative_primary,R,Roughly traced,cb_intrinsic,None,...,None,early,NaN,NaN,None,None,None,"[41190, 18491, 12885]",None,Traced
10006,10006,"VS1,VS2,VS3,VS4,VS5,VS6,VS7,VS8",10006.0,VS,VS_L,no_lineage,L,Prelim Roughly traced,visual_projection,10006,...,None,None,NaN,NaN,None,None,None,"[69604, 28368, 42459]",None,Traced


# Local data server

In the rest of this notebook, we'll create files to load in neuroglancer.

To actually used them, we must host them via http somehow.  One option is to upload them to a Google Storage Bucket, or to host them on your own web server.

Just for convenient development and debug, ngsidekick provides an http server that will let you browse files on your own machine.

In [29]:
server_port = 9091
server_proc, server_address, log_file = ngsk.serve_directory(TUTORIAL_EXPORTS, port=server_port, background=True, log_to_file=True)
TUTORIAL_DATA_ADDRESS = server_address
print(server_address)

# Note: Call server_proc.terminate() to stop the server.

http://10.101.10.33:9091


### Tip: Janelia users have the option of using FileGlancer to host their data!

https://fileglancer.int.janelia.org

In [13]:
# Only run this cell if you're using FileGlancer instead of the local server above!

# Data link, copied from the FileGlancer website.
TUTORIAL_DATA_ADDRESS = 'https://fileglancer.int.janelia.org/fc/files/MgV-YDzs9gEEwzTR/ngsidekick-tutorial-exports'

# Segment Properties

Neuroglancer can display a label (name) next to the segment IDs in the side panel.

The names must come from a "segment properties" source.  Let's create our own.

In [30]:
# Just for illustration, we'll add a suffix to the type name to indicate if the cell has a soma or not.
new_df = df.query('cell_type.notnull()').copy()
df['mylabel'] = df['cell_type']
df.loc[df['somaLocation'].notnull(), 'mylabel'] += ' (has soma)'

# Write the new segment properties.
# We must place them in a file named 'info' in an otherwise empty directory.
info = ngsk.segment_properties_json(df[['mylabel']], 'mylabel', output_path=f'{TUTORIAL_EXPORTS}/mysegprops/info')

You can view the new segment properties by manually adding a new "source" to the segmentation layer in your neuroglancer view.

Here's how to do it programmatically.

In [31]:
new_state = copy.deepcopy(state)
seg_layer = ngsk.layer_state(new_state, 'cns-seg')

# Don't use the properties provided in the original data;
# we want to use our own.
if 'properties' in seg_layer['source']['subsources']:
    del seg_layer['source']['subsources']['properties']

# Convert 'source' to a list so we can add a second entry.
if not isinstance(seg_layer['source'], list):
    seg_layer['source'] = [seg_layer['source']]

# Add our new source, using the local server address.
seg_layer['source'].append(f'precomputed://{TUTORIAL_DATA_ADDRESS}/mysegprops')

# Note: If you're using a local data server and you don't refer to it via 'localhost',
# then you can't use https here.  We use bare http instead.
ngsk.encode_ngstate('http://neuroglancer-demo.appspot.com', new_state)

'http://neuroglancer-demo.appspot.com/#!%7B%22title%22%3A%20%22Example%20Scene%201%22%2C%20%22dimensions%22%3A%20%7B%22x%22%3A%20%5B8e-09%2C%20%22m%22%5D%2C%20%22y%22%3A%20%5B8e-09%2C%20%22m%22%5D%2C%20%22z%22%3A%20%5B8e-09%2C%20%22m%22%5D%7D%2C%20%22position%22%3A%20%5B48311.5%2C%2025295.765625%2C%2028511.5%5D%2C%20%22crossSectionScale%22%3A%2088.13122564876332%2C%20%22projectionOrientation%22%3A%20%5B0%2C%20-0.002181659685447812%2C%200%2C%200.999997615814209%5D%2C%20%22projectionScale%22%3A%2099656.51830908704%2C%20%22layers%22%3A%20%5B%7B%22type%22%3A%20%22image%22%2C%20%22source%22%3A%20%7B%22url%22%3A%20%22precomputed%3A//gs%3A//flyem-male-cns/em/em-clahe-jpeg%22%2C%20%22subsources%22%3A%20%7B%22default%22%3A%20true%7D%2C%20%22enableDefaultSubsources%22%3A%20false%7D%2C%20%22tab%22%3A%20%22rendering%22%2C%20%22name%22%3A%20%22em-clahe%22%7D%2C%20%7B%22type%22%3A%20%22segmentation%22%2C%20%22source%22%3A%20%5B%7B%22url%22%3A%20%22precomputed%3A//gs%3A//flyem-male-cns/v0.9/segmentat

Request: mysegprops/info [no range]
Response: 200 for mysegprops/info
INFO:werkzeug:10.101.10.33 - - [04/Dec/2025 12:41:03] "GET /mysegprops/info HTTP/1.1" 200 -


### Segment properties: Tags

You can also add 'tags' to your segment properties.

In [32]:
new_df = df.copy()

# Tags can be created from boolean columns (true entries get the tag)
# or from string/categorical columns (in which case the tag name is prefixed with the column name)
df['has_soma'] = df['somaLocation'].notnull()

info = ngsk.segment_properties_json(
    df[['cell_type', 'superclass', 'has_soma']],
    label_col='cell_type',
    tag_cols=['superclass', 'has_soma'],
    output_path=f'{TUTORIAL_EXPORTS}/mysegprops/info'
)

# Now refresh your browser to see the new tags we exported.

# Point Annotations

Neuroglancer can also show various types of vector annotations: points, lines, boxes, ellipsoids.

NGSidekick comes with a function for constructing such annotations from a DataFrame.

We'll use the MaleCNS soma locations for this example.


In [33]:
# Extract the rows with a soma location
soma_df = df.dropna(subset='somaLocation').copy()
soma_df[[*'xyz']] = soma_df['somaLocation'].tolist()

print(soma_df.shape)
soma_df[['bodyId', 'cell_type', *'xyz']].head()


(141780, 40)


,bodyId,cell_type,x,y,z
segment,,,,,
10001,10001,DNp01,37124,22258,36274
10002,10002,OCG01d,50214,10279,27497
10003,10003,VCH,46716,21029,15714
10005,10005,AOTU019,41190,18491,12885
10006,10006,VS,69604,28368,42459


# Local Annotations
To visualize just a few points, neuroglancer allows "local" annotations -- stored directly in the neuroglancer link (JSON state).

In [34]:
# Neuroglancer allows local annotation layers to contain a mix of annotation types (points, lines, boxes, etc.)
Tm9_soma_df = soma_df.query('cell_type == "Tm9"')
Tm9_soma_layer_json = ngsk.local_annotation_json(Tm9_soma_df, res_nm_xyz=(8, 8, 8))

# The output below can be pasted into the neuroglancer state.
print(json.dumps(Tm9_soma_layer_json))

{"name": "annotations", "type": "annotation", "source": {"url": "local://annotations", "transform": {"outputDimensions": {"x": [8e-09, "m"], "y": [8e-09, "m"], "z": [8e-09, "m"]}}}, "tool": "annotatePoint", "shader": "void main() {\n    //\n    // Point Marker API\n    //\n    setPointMarkerSize(8.0);\n    setPointMarkerColor(vec4(defaultColor(), 1.0));\n    setPointMarkerBorderWidth(1.0);\n    setPointMarkerBorderColor(defaultColor());\n}\n", "annotations": [{"type": "point", "id": "0xc9f82d33fad0444", "point": [75339, 18690, 24056]}, {"type": "point", "id": "0x12e5618c157d9ce6", "point": [82763, 46908, 29906]}, {"type": "point", "id": "0x3a3c9e43051c2a98", "point": [81099, 40221, 23977]}, {"type": "point", "id": "0x18a3b68ca416b46d", "point": [76408, 21362, 22562]}, {"type": "point", "id": "0x3b4f695c3e2bba7c", "point": [89208, 18548, 33472]}, {"type": "point", "id": "0x606614a796f2b947", "point": [78398, 33256, 20886]}, {"type": "point", "id": "0x51d9ffc74ada8e46", "point": [91380, 

In [35]:
# Or add it programmatically to an existing neuroglancer state.
new_state = copy.deepcopy(state)
new_state['layers'].append(Tm9_soma_layer_json)
print(ngsk.encode_ngstate('http://neuroglancer-demo.appspot.com', new_state))

http://neuroglancer-demo.appspot.com/#!%7B%22title%22%3A%20%22Example%20Scene%201%22%2C%20%22dimensions%22%3A%20%7B%22x%22%3A%20%5B8e-09%2C%20%22m%22%5D%2C%20%22y%22%3A%20%5B8e-09%2C%20%22m%22%5D%2C%20%22z%22%3A%20%5B8e-09%2C%20%22m%22%5D%7D%2C%20%22position%22%3A%20%5B48311.5%2C%2025295.765625%2C%2028511.5%5D%2C%20%22crossSectionScale%22%3A%2088.13122564876332%2C%20%22projectionOrientation%22%3A%20%5B0%2C%20-0.002181659685447812%2C%200%2C%200.999997615814209%5D%2C%20%22projectionScale%22%3A%2099656.51830908704%2C%20%22layers%22%3A%20%5B%7B%22type%22%3A%20%22image%22%2C%20%22source%22%3A%20%7B%22url%22%3A%20%22precomputed%3A//gs%3A//flyem-male-cns/em/em-clahe-jpeg%22%2C%20%22subsources%22%3A%20%7B%22default%22%3A%20true%7D%2C%20%22enableDefaultSubsources%22%3A%20false%7D%2C%20%22tab%22%3A%20%22rendering%22%2C%20%22name%22%3A%20%22em-clahe%22%7D%2C%20%7B%22type%22%3A%20%22segmentation%22%2C%20%22source%22%3A%20%7B%22url%22%3A%20%22precomputed%3A//gs%3A//flyem-male-cns/v0.9/segmentation%

In [36]:
long_link = ngsk.encode_ngstate('https://neuroglancer-demo.appspot.com', state)

import requests
SHORTENER_URL = 'https://shortng-bmcp5imp6q-uc.a.run.app/shortng'
r = requests.post(f'{SHORTENER_URL}', json={'state': new_state})
r.content

b'{"link":"https://neuroglancer-demo.appspot.com/#!gs://flyem-user-links/short/2025-12-04.174130.759439.json"}\n'

# Precomputed Annotations

If you're dealing with lots of annotations (more than a few thousand), you'll need to export them in neuroglancer "precomputed" format.

In [ ]:
# Fill empty values with a category that will correspond to code 0.
soma_df['itoleeHl'] = soma_df['itoleeHl'].fillna('').astype('category')
soma_df['trumanHl'] = soma_df['trumanHl'].fillna('').astype('category')

coord_space = {
    "names": ['x', 'y', 'z'],
    "units": ['nm', 'nm', 'nm'],
    "scales": [8, 8, 8]
}

ngsk.write_precomputed_annotations(
    soma_df,
    coord_space,
    'point',
    properties=['itoleeHl', 'trumanHl'],
    relationships=['bodyId'],
    output_dir=f'{TUTORIAL_EXPORTS}/soma_annotations',
)

[2025-12-04 12:41:39,566] INFO Encoding annotation IDs
[2025-12-04 12:41:39,575] INFO Encoding annotation geometries and properties
[2025-12-04 12:41:39,590] INFO Encoding relationships
[2025-12-04 12:41:39,610] INFO Writing annotations to 'by_id' index


100%|██████████| 141780/141780 [00:00<00:00, 1009284.52it/s]


[2025-12-04 12:41:40,435] INFO Grouping annotations by relationship bodyId
[2025-12-04 12:41:41,543] INFO Combining annotation and ID buffers for relationship 'bodyId'
[2025-12-04 12:41:41,572] INFO Writing annotations to 'by_rel_bodyId' index


100%|██████████| 141780/141780 [00:00<00:00, 1042575.57it/s]


[2025-12-04 12:41:42,468] INFO Shuffling annotations before assigning spatial grid levels
[2025-12-04 12:41:42,473] INFO Assigning spatial grid chunks
[2025-12-04 12:41:42,597] INFO Done assigning spatial grid chunks
[2025-12-04 12:41:42,597] INFO Concatenating annotations by spatial chunk
[2025-12-04 12:41:42,619] INFO Combining annotation and ID buffers for spatial index
[2025-12-04 12:41:42,621] INFO Writing annotations to 'by_spatial_level_0' index


...


100%|██████████| 2/2 [00:00<00:00, 32768.00it/s]

[2025-12-04 12:41:42,746] INFO Writing annotations to 'by_spatial_level_2' index



100%|██████████| 4/4 [00:00<00:00, 28008.71it/s]


[2025-12-04 12:41:42,911] INFO Writing annotations to 'by_spatial_level_3' index


100%|██████████| 12/12 [00:00<00:00, 75009.91it/s]


In [38]:
# Or add it programmatically to an existing neuroglancer state.
new_state = copy.deepcopy(state)
new_state['layers'].append({
    'name': 'soma_points',
    'source': f'precomputed://{TUTORIAL_DATA_ADDRESS}/soma_annotations'
})
print(ngsk.encode_ngstate('http://neuroglancer-demo.appspot.com', new_state))

http://neuroglancer-demo.appspot.com/#!%7B%22title%22%3A%20%22Example%20Scene%201%22%2C%20%22dimensions%22%3A%20%7B%22x%22%3A%20%5B8e-09%2C%20%22m%22%5D%2C%20%22y%22%3A%20%5B8e-09%2C%20%22m%22%5D%2C%20%22z%22%3A%20%5B8e-09%2C%20%22m%22%5D%7D%2C%20%22position%22%3A%20%5B48311.5%2C%2025295.765625%2C%2028511.5%5D%2C%20%22crossSectionScale%22%3A%2088.13122564876332%2C%20%22projectionOrientation%22%3A%20%5B0%2C%20-0.002181659685447812%2C%200%2C%200.999997615814209%5D%2C%20%22projectionScale%22%3A%2099656.51830908704%2C%20%22layers%22%3A%20%5B%7B%22type%22%3A%20%22image%22%2C%20%22source%22%3A%20%7B%22url%22%3A%20%22precomputed%3A//gs%3A//flyem-male-cns/em/em-clahe-jpeg%22%2C%20%22subsources%22%3A%20%7B%22default%22%3A%20true%7D%2C%20%22enableDefaultSubsources%22%3A%20false%7D%2C%20%22tab%22%3A%20%22rendering%22%2C%20%22name%22%3A%20%22em-clahe%22%7D%2C%20%7B%22type%22%3A%20%22segmentation%22%2C%20%22source%22%3A%20%7B%22url%22%3A%20%22precomputed%3A//gs%3A//flyem-male-cns/v0.9/segmentation%

Let's add an annotation shader that uses the properties.

In [39]:
SHADER = """\
#uicontrol float radius slider(min=1, max=30, step=1, default=5)

uint hash_function(uint state, uint value) {
    uint k1 = 0xcc9e2d51u;
    uint k2 = 0x1b873593u;

    value = value * k1;
    value = (value << 15) | (value >> 17);
    value = value * k2;
    state = state ^ value;
    state = (state << 13) | (state >> 19);
    state = state * 5u + 0xe6546b64u;

    return state;
}

vec3 hsv_to_rgb(float h, float s, float v) {
    h *= 6.0;
    float hue_index = floor(h);
    float remainder = h - hue_index;
    float val1 = v * (1.0 - s);
    float val2 = v * (1.0 - (s * remainder));
    float val3 = v * (1.0 - (s * (1.0 - remainder)));

    int hue_remainder = int(hue_index) % 6;

    if (hue_remainder == 0) {
        return vec3(v, val3, val1);
    } else if (hue_remainder == 1) {
        return vec3(val2, v, val1);
    } else if (hue_remainder == 2) {
        return vec3(val1, v, val3);
    } else if (hue_remainder == 3) {
        return vec3(val1, val2, v);
    } else if (hue_remainder == 4) {
        return vec3(val3, val1, v);
    } else { // hue_remainder == 5
        return vec3(v, val1, val2);
    }
}

vec3 color_from_segment_id(uint color_seed_, uint segment_id) {
    uint result = hash_function(color_seed_, segment_id);

    // We only support 32-bit segment IDs, so newvalue is always 0,
    // and (id >> 32) would be undefined behavior!
    // uint newvalue = id >> 32;

    uint newvalue = 0u;
    uint result2 = hash_function(result, newvalue);

    float c0 = float(result2 & 0xFFu) / 255.0;
    float c1 = float((result2 >> 8) & 0xFFu) / 255.0;

    float h = c0;
    float s = 0.5 + 0.5 * c1;
    float v = 1.0;

    return hsv_to_rgb(h, s, v);
}

void main() {
    setPointMarkerSize(radius);
    setPointMarkerBorderWidth(1.0);
    setPointMarkerBorderColor(vec3(0.0, 0.0, 0.0));

    setColor(vec3(1.0, 1.0, 1.0));

    if (int(prop_itoleeHl()) > 0) {
      setColor(color_from_segment_id(uint(0), uint(prop_itoleeHl())));
    }
    if (int(prop_trumanHl()) > 0) {
      setColor(color_from_segment_id(uint(0), uint(prop_trumanHl())));
    }
}
"""

# Or add it programmatically to an existing neuroglancer state.
new_state = copy.deepcopy(state)
new_state['layers'].append({
    'type': 'annotation',
    'name': 'soma_points',
    'source': f'precomputed://{TUTORIAL_DATA_ADDRESS}/soma_annotations',
    'shader': SHADER
})
print(ngsk.encode_ngstate('http://neuroglancer-demo.appspot.com', new_state))

http://neuroglancer-demo.appspot.com/#!%7B%22title%22%3A%20%22Example%20Scene%201%22%2C%20%22dimensions%22%3A%20%7B%22x%22%3A%20%5B8e-09%2C%20%22m%22%5D%2C%20%22y%22%3A%20%5B8e-09%2C%20%22m%22%5D%2C%20%22z%22%3A%20%5B8e-09%2C%20%22m%22%5D%7D%2C%20%22position%22%3A%20%5B48311.5%2C%2025295.765625%2C%2028511.5%5D%2C%20%22crossSectionScale%22%3A%2088.13122564876332%2C%20%22projectionOrientation%22%3A%20%5B0%2C%20-0.002181659685447812%2C%200%2C%200.999997615814209%5D%2C%20%22projectionScale%22%3A%2099656.51830908704%2C%20%22layers%22%3A%20%5B%7B%22type%22%3A%20%22image%22%2C%20%22source%22%3A%20%7B%22url%22%3A%20%22precomputed%3A//gs%3A//flyem-male-cns/em/em-clahe-jpeg%22%2C%20%22subsources%22%3A%20%7B%22default%22%3A%20true%7D%2C%20%22enableDefaultSubsources%22%3A%20false%7D%2C%20%22tab%22%3A%20%22rendering%22%2C%20%22name%22%3A%20%22em-clahe%22%7D%2C%20%7B%22type%22%3A%20%22segmentation%22%2C%20%22source%22%3A%20%7B%22url%22%3A%20%22precomputed%3A//gs%3A//flyem-male-cns/v0.9/segmentation%